<a href="https://colab.research.google.com/github/Murtazali05/Recommendation-System-platform/blob/main/RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Restricted Bolzman machine**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab\ Notebooks/Recommendation-System

/content/drive/MyDrive/Colab Notebooks/Recommendation-System


In [3]:
#Importing the libraries

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [4]:
from read_data import read_data, create_dataset,create_tabular_data

In [5]:
users = pd.read_csv("ml-1m/users.dat",sep = "::", header = None, engine = "python",encoding ="latin-1")

In [6]:
from pathlib import Path
ratings, movies = read_data(Path('ml-1m'))

In [7]:
nb_users = len(users)
nb_movies = len(movies)

In [9]:
nb_movies

3883

In [10]:
nb_users

6040

In [ ]:
df = create_tabular_data(ratings, users, movies) 

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2)

In [ ]:
train_set = torch.FloatTensor(train_set.to_numpy())
test_set = torch.FloatTensor(test_set.to_numpy())

In [ ]:

# Converting the ratings into binary ratings 1(liked) and 0(disliked)
train_set[train_set==0] = -1
train_set[train_set==1] = 0
train_set[train_set==2] = 0
train_set[train_set>=3] = 1
test_set[test_set==0] = -1
test_set[test_set==1] = 0
test_set[test_set==2] = 0
test_set[test_set>=3] = 1


In [ ]:
from rbm import RBM

In [ ]:
# from torch import nn
# # Creating the architecture of neural network
# class RBM(nn.Module):

#     def __init__(self,nv,nh):
#         super().__init__()
#         self.W = torch.randn(nh,nv)
#         self.a = torch.randn(1,nh)
#         self.b = torch.randn(1,nv)
        
#     def sample_h(self,x):
#         wx = torch.mm(x,self.W.t())
#         activation = wx+self.a.expand_as(wx)
#         p_h_given_v = torch.sigmoid(activation)
#         return p_h_given_v, torch.bernoulli(p_h_given_v)
#     def sample_v(self,y):
#         wy = torch.mm(y,self.W)
#         activation = wy+self.b.expand_as(wy)
#         p_v_given_h = torch.sigmoid(activation)
#         return p_v_given_h, torch.bernoulli(p_v_given_h)
#     def train(self, v0, vk, ph0, phk):
#         self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
#         self.b += torch.sum((v0-vk), 0)
#         self.a += torch.sum((ph0-phk), 0)
        

In [ ]:
  
nv = len(train_set[0])
nh = 100
batch_size = 500



In [ ]:
rbm = RBM(nv,nh)



In [ ]:

#Training the RBM
nb_epochs = 10
nb_users = train_set.shape[0]
for epoch in range(1, nb_epochs+1):
    # import pdb;pdb.set_trace()
    train_loss = 0
    s = float(0)
    for id_user in range(0,nb_users-batch_size,batch_size):
        vk = train_set[id_user:id_user+batch_size]
        v0 = train_set[id_user:id_user+batch_size]
        ph0,_=rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)   
            _, vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_= rbm.sample_h(vk)
        rbm.train(v0,vk,ph0,phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
        s += 1.
        # print(train_loss)
        
        
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
        

In [ ]:
import copy
best_weights = copy.deepcopy(rbm.state_dict())


In [ ]:
# import pickle
# with open('RBM.weights', 'wb') as file:
#     pickle.dump(best_weights, file)

In [ ]:
import pickle
fh = open('RBM.weights','rb')
weights = pickle.load(fh)
rbm.load_state_dict(weights)
fh.close()

In [ ]:
rbm.load_state_dict(weights)


In [ ]:

#Testing the RBM

test_loss = 0
s = float(0)
nb_users = test_set.shape[1]
for id_user in range(nb_users):
    
    v = train_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)   
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0]-v[vt>=0]))
        s += 1.
print(' loss: '+str(test_loss/s))
            

In [ ]:
import random

In [ ]:
id_user=random.randint(0,test_set.shape[0])
v = train_set[id_user:id_user+1]
vt = test_set[id_user:id_user+1]

_, h = rbm.sample_h(v)   
_, v = rbm.sample_v(h)


In [ ]:
id_user

In [ ]:
prediction_df = pd.DataFrame(v.numpy().reshape(-1,1),columns=['movie_id'])

In [ ]:
random.choices(list(prediction_df[prediction_df['movie_id']>0].index),k=5)

In [ ]:
def predict_recommendations(user_id,model,N=5):
    v=unseen_set[id_user:id_user+1]
    _, h = model.sample_h(v)   
    _, v = model.sample_v(h)
    prediction_df = pd.DataFrame(v.numpy().reshape(-1,1),columns=['movie_id'])
    potential_likes = prediction_df[prediction_df['movie_id']>0].index
    recommendations = list(potential_likes)
    return random.choices(recommendations,k=N)


In [ ]:
import random

In [ ]:
user_id = random.randint(0,test_set.shape[0])
print("predicting recommendations for user: {}".format(user_id))
v=test_set[user_id:user_id+1]
_, h = rbm.sample_h(v)   
_, v = rbm.sample_v(h)
out_df = pd.DataFrame(v.numpy()[0],columns=['predicted_ratings'])
out_df['ground_truth'] = test_set[user_id:user_id+1].numpy()[0]
rec_df = out_df[out_df['ground_truth']==-1][out_df['predicted_ratings']==1]
random.choices(rec_df.index,k=N)

In [ ]:
out_df

In [ ]:
import random
random.choices(rec_df.index,k=5)

In [ ]:

# filtering unwatched movies and sorting according to the predicted rating
rec_df = out_df[out_df['ground_truth']==-1 and out_df['predicted_ratings']==1].sort_values(by='predicted_ratings',ascending=False)
recs = [x+1 for x in rec_df[:4*N].index]
return random.sample(recs,k=N)
# prediction_df = pd.DataFrame(v.numpy().reshape(-1,1),columns=['movie_id'])
# potential_likes = prediction_df[prediction_df['movie_id']>0].index
# recommendations = list(potential_likes)
# return random.choices(recommendations,k=N)